# Import

In [1]:
import os
import pandas as pd
import random
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import matplotlib.pyplot as plt

import glob
import os
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) 

# Data Load

In [2]:
train_df = pd.read_csv('../open/train.csv').drop(columns='ID')
test_df = pd.read_csv('../open/test.csv').drop(columns='ID')

In [3]:
display(train_df)
print(train_df.shape)

,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,승용,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,...,상해없음,승용,남,52세,경상,0,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,2021-12-31 19,금요일,맑음,대구광역시 수성구 수성동3가,교차로 - 교차로안,건조,차대차,측면충돌,신호위반,승용,...,상해없음,이륜,남,28세,경상,0,0,1,0,3
39605,2021-12-31 19,금요일,맑음,대구광역시 달서구 상인동,단일로 - 기타,건조,차대차,측면충돌,안전거리미확보,승용,...,상해없음,승용,남,52세,경상,0,0,1,0,3
39606,2021-12-31 21,금요일,맑음,대구광역시 달서구 월성동,교차로 - 교차로안,건조,차대차,측면충돌,교차로운행방법위반,승용,...,중상,승용,남,73세,중상,0,2,0,0,10
39607,2021-12-31 22,금요일,맑음,대구광역시 달서구 장동,기타 - 기타,건조,차대차,추돌,안전운전불이행,승용,...,상해없음,승용,여,57세,경상,0,0,1,0,3


(39609, 22)


In [4]:
display(test_df)
print(test_df.shape)

,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
0,2022-01-01 01,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람
1,2022-01-01 01,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람
2,2022-01-01 04,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차
3,2022-01-01 04,토요일,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차
4,2022-01-01 06,토요일,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차
...,...,...,...,...,...,...,...
10958,2022-12-31 18,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차
10959,2022-12-31 18,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차
10960,2022-12-31 20,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차
10961,2022-12-31 20,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차


(10963, 7)


# Data Preprocessing

In [5]:
train_df = train_df.drop(columns=['부상자수','경상자수','중상자수','사망자수','피해운전자 상해정도','피해운전자 연령','피해운전자 차종','가해운전자 상해정도',
                           '가해운전자 연령','가해운전자 성별','가해운전자 차종','법규위반','사고유형 - 세부분류','피해운전자 성별'])
print(train_df.shape)

(39609, 8)


In [6]:
train_df['구'] = train_df['시군구'].str.split(" ").str[1]
test_df['구'] = test_df['시군구'].str.split(" ").str[1]

In [7]:
train_df['동'] = train_df['시군구'].str.split(" ").str[2]
test_df['동'] = test_df['시군구'].str.split(" ").str[2]

In [8]:
train_df.drop(['시군구'],axis=1,inplace=True)
test_df.drop(['시군구'],axis=1,inplace=True)

In [9]:
train_df['도로형태1'] = train_df['도로형태'].str.split(" ").str[0]
test_df['도로형태1'] = test_df['도로형태'].str.split(" ").str[0]
train_df['도로형태2'] = train_df['도로형태'].str.split(" ").str[2]
test_df['도로형태2'] = test_df['도로형태'].str.split(" ").str[2]
train_df.drop(['도로형태'],axis=1,inplace=True)
test_df.drop(['도로형태'],axis=1,inplace=True)

In [10]:
def feat_eng(df):
    # datetime 변환
    df['사고일시'] = pd.to_datetime(df['사고일시'])
    
    #df['년'] = df['사고일시'].dt.year
    df['월'] = df['사고일시'].dt.month
    df['일'] = df['사고일시'].dt.day
    df['시'] = df['사고일시'].dt.hour
    
    # 불필요 컬럼 제거
    subs = ['사고일시']
    df = df.drop(subs, axis=1)
    
    return df

In [11]:
train_df = feat_eng(train_df)
test_df = feat_eng(test_df)

In [12]:
display(train_df)
print(train_df.shape)

,요일,기상상태,노면상태,사고유형,ECLO,구,동,도로형태1,도로형태2,월,일,시
0,화요일,맑음,건조,차대사람,5,중구,대신동,단일로,기타,1,1,0
1,화요일,흐림,건조,차대사람,3,달서구,감삼동,단일로,기타,1,1,0
2,화요일,맑음,건조,차대사람,3,수성구,두산동,단일로,기타,1,1,1
3,화요일,맑음,건조,차대차,5,북구,복현동,단일로,기타,1,1,2
4,화요일,맑음,건조,차대차,3,동구,신암동,단일로,기타,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
39604,금요일,맑음,건조,차대차,3,수성구,수성동3가,교차로,교차로안,12,31,19
39605,금요일,맑음,건조,차대차,3,달서구,상인동,단일로,기타,12,31,19
39606,금요일,맑음,건조,차대차,10,달서구,월성동,교차로,교차로안,12,31,21
39607,금요일,맑음,건조,차대차,3,달서구,장동,기타,기타,12,31,22


(39609, 12)


In [13]:
display(test_df)
print(test_df.shape)

,요일,기상상태,노면상태,사고유형,구,동,도로형태1,도로형태2,월,일,시
0,토요일,맑음,건조,차대사람,수성구,상동,교차로,교차로안,1,1,1
1,토요일,맑음,건조,차대사람,수성구,지산동,단일로,기타,1,1,1
2,토요일,맑음,건조,차대차,수성구,수성동2가,교차로,교차로안,1,1,4
3,토요일,맑음,건조,차대차,수성구,신매동,단일로,기타,1,1,4
4,토요일,맑음,건조,차대차,달서구,감삼동,교차로,교차로안,1,1,6
...,...,...,...,...,...,...,...,...,...,...,...
10958,토요일,맑음,건조,차대차,남구,대명동,단일로,터널,12,31,18
10959,토요일,맑음,건조,차대차,수성구,시지동,단일로,기타,12,31,18
10960,토요일,맑음,건조,차대차,수성구,연호동,단일로,기타,12,31,20
10961,토요일,맑음,건조,차대차,수성구,범물동,교차로,교차로부근,12,31,20


(10963, 11)


In [14]:
train = train_df.copy()
test = test_df.copy()

In [15]:
# Categorical 컬럼 인코딩
categorical_features = test.columns


for feature in tqdm(categorical_features, desc="Encoding features"):
    encoder = LabelEncoder()
    train[feature] = encoder.fit_transform(train[feature])
    for label in np.unique(test[feature]):
        if label not in encoder.classes_:
            encoder.classes_ = np.append(encoder.classes_, label)
    test[feature] = encoder.transform(test[feature])

Encoding features: 100%|███████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 93.61it/s]


In [16]:
train

,요일,기상상태,노면상태,사고유형,ECLO,구,동,도로형태1,도로형태2,월,일,시
0,6,2,0,0,5,7,40,2,5,0,0,0
1,6,5,0,0,3,1,4,2,5,0,0,0
2,6,2,0,0,3,6,66,2,5,0,0,1
3,6,2,0,1,5,4,79,2,5,0,0,2
4,6,2,0,1,3,3,129,2,5,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...
39604,0,2,0,1,3,6,118,0,3,11,30,19
39605,0,2,0,1,3,1,103,2,5,11,30,19
39606,0,2,0,1,10,1,144,0,3,11,30,21
39607,0,2,0,1,3,1,158,1,5,11,30,22


In [17]:
test

,요일,기상상태,노면상태,사고유형,구,동,도로형태1,도로형태2,월,일,시
0,5,2,0,0,6,99,0,3,0,0,1
1,5,2,0,0,6,168,2,5,0,0,1
2,5,2,0,1,6,117,0,3,0,0,4
3,5,2,0,1,6,126,2,5,0,0,4
4,5,2,0,1,1,4,0,3,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...
10958,5,2,0,1,0,38,2,9,11,30,18
10959,5,2,0,1,6,123,2,5,11,30,18
10960,5,2,0,1,6,134,2,5,11,30,20
10961,5,2,0,1,6,77,0,2,11,30,20


In [24]:
train.요일.unique()

array([6, 2, 1, 0, 5, 4, 3])

In [23]:
test.요일.unique()

array([5, 4, 3, 6, 2, 1, 0])

# Autogluon

In [18]:
train_data = TabularDataset(train)
test_data = TabularDataset(test)

In [19]:
label = 'ECLO'
eval_metric = 'root_mean_squared_error'

In [20]:
predictor = TabularPredictor(
    label=label, problem_type='regression', eval_metric=eval_metric
).fit(train_data, 
      presets='best_quality', 
      num_stack_levels=3,
      #excluded_model_types = excluded_model_types,
      num_gpus=1)

No path specified. Models will be saved in: "AutogluonModels\ag-20231121_122122\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231121_122122\"
AutoGluon Version:  0.8.2
Python Version:     3.9.18
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   228.08 GB / 999.46 GB (22.8%)
Train Data Rows:    39609
Train Data Columns: 11
Label Column: ECLO
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    57550.24 MB
	Train Data (Original)  Memory Usage: 2.22 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...


In [21]:
predictor.leaderboard(silent = True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-3.153085,6.083269,1061.986070,0.001998,0.540702,3,True,22
1,WeightedEnsemble_L4,-3.153262,8.280485,1352.213945,0.001002,0.580027,4,True,32
2,CatBoost_BAG_L3,-3.153993,7.736693,1189.295336,0.029499,56.098062,3,True,26
3,WeightedEnsemble_L5,-3.154292,11.961173,1911.110930,0.000991,0.537604,5,True,42
4,CatBoost_BAG_L2,-3.154614,3.843601,621.271324,0.031920,36.092016,2,True,16
5,NeuralNetFastAI_BAG_L3,-3.154654,8.136472,1274.317578,0.429277,141.120303,3,True,28
6,CatBoost_BAG_L4,-3.154787,11.474177,1749.842058,0.023058,49.164867,4,True,36
7,WeightedEnsemble_L2,-3.155475,1.938660,270.555804,0.001001,0.611487,2,True,12
8,NeuralNetFastAI_BAG_L4,-3.155657,11.873075,1841.670030,0.421956,140.992839,4,True,38
9,LightGBMXT_BAG_L3,-3.155931,7.780755,1138.780706,0.073560,5.583431,3,True,23


In [25]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test, model=model_to_use)

In [26]:
sample_submission = pd.read_csv('../open/sample_submission.csv')
baseline_submission = sample_submission.copy()
baseline_submission['ECLO'] = model_pred.astype(int)
baseline_submission

,ID,ECLO
0,ACCIDENT_39609,4
1,ACCIDENT_39610,3
2,ACCIDENT_39611,5
3,ACCIDENT_39612,5
4,ACCIDENT_39613,5
...,...,...
10958,ACCIDENT_50567,6
10959,ACCIDENT_50568,4
10960,ACCIDENT_50569,5
10961,ACCIDENT_50570,5


In [27]:
baseline_submission.to_csv('../Sub/autogluon_preprocessing.csv', index=False)